# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

In [2]:
#load base model

'''
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(
    "gpt2",
    num_labels=2,
    id2label={0: "negative", 1: "positive"},
    label2id={"negative": 0, "positive": 1},

)
'''

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label={0: "negative", 1: "positive"},
    label2id={"negative": 0, "positive": 1},
)

#freeze base model parameters
for param in model.base_model.parameters():
    param.requires_grad = False
    
#contrast trainable parameter count between lORA vs base model
print_trainable_parameters(model)
#print_trainable_parameters(lora_model)
model

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 592130 || all params: 66955010 || trainable%: 0.88


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [3]:
#load data and pre-process data
from datasets import load_dataset

# Load the train and test splits of the rotten_tomatoes dataset
splits = ["train", "test"]
dataset = {split: ds for split, ds in zip(splits, load_dataset("rotten_tomatoes", split=splits))}
#dataset = load_dataset("rotten_tomatoes", split=splits)


# Thin out the dataset to make it run faster for this example
for split in splits:
    dataset[split] = dataset[split].shuffle(seed=42).select(range(500))
# Show the dataset

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    """Preprocess the dataset by returning tokenized examples."""
    return tokenizer(examples["text"],return_tensors='pt', padding=True, truncation=True)

tokenized_ds = {}
for split in splits:
    tokenized_ds[split] = dataset[split].map(preprocess_function, batched=True)


tokenized_ds
#tokenized_ds['train'][0]

print_trainable_parameters(model)

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

trainable params: 592130 || all params: 66955010 || trainable%: 0.88


In [4]:
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

# Prepare your evaluation dataset
eval_dataset = tokenized_ds["test"]
train_dataset = tokenized_ds["train"]
# Define the training arguments

'''
training_args = TrainingArguments(
    output_dir='./data/rotten_tomatos',
    evaluation_strategy='epoch',
    per_device_eval_batch_size=4,
)
'''
training_args = TrainingArguments(
    output_dir='./data/rotten_tomatos',
    learning_rate=1e-3,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Create the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=eval_dataset,
    train_dataset = train_dataset,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)


#train the model
trainer.train()

# Run the evaluation
evaluation = trainer.evaluate()

# Print the evaluation metrics
print(evaluation)
print_trainable_parameters(model)

/opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.562928,0.678000
2,No log,0.526752,0.730000


Checkpoint destination directory ./data/rotten_tomatos/checkpoint-42 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/rotten_tomatos/checkpoint-84 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'eval_loss': 0.5267519950866699, 'eval_accuracy': 0.73, 'eval_runtime': 108.0929, 'eval_samples_per_second': 4.626, 'eval_steps_per_second': 0.389, 'epoch': 2.0}
trainable params: 592130 || all params: 66955010 || trainable%: 0.88


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [6]:
#train peft model
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments
import numpy as np

#PEFT configuration steps
from peft import LoraConfig

config = LoraConfig(
    r = 12, 
    lora_alpha = 12,
    bias = 'lora_only',
    target_modules=["v_lin"],
    #use_rslora = True,
    modules_to_save=["decode_attn"],
)

#get trainable peft model and Wrap the base model with get_peft_model() 
#to get a trainable PeftModel.
from peft import get_peft_model
lora_model = get_peft_model(model, config)
print_trainable_parameters(lora_model)

'''
training_args = TrainingArguments(
    output_dir='./data/rotten_tomatos_lora',
    learning_rate=1e-3,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)
'''

train_lora = tokenized_ds['train'].rename_column('label', 'labels')
test_lora = tokenized_ds['test'].rename_column('label', 'labels')

train_lora.set_format('torch',columns=['labels', 'input_ids', 'attention_mask'])
test_lora.set_format('torch',columns=['labels', 'input_ids', 'attention_mask'])

training_args = TrainingArguments(
    output_dir='./data/rotten_tomatos_lora',
    label_names = ["start_positions", "end_positions"],
    evaluation_strategy='epoch',
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    num_train_epochs=2,
    save_strategy="epoch",
    load_best_model_at_end=True,
    remove_unused_columns=False,
)


# Create the trainer
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset = train_lora,
    eval_dataset = test_lora,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

#train the model
trainer.train()

trainable params: 115200 || all params: 67065602 || trainable%: 0.17


Epoch,Training Loss,Validation Loss
1,No log,No log


KeyError: 'eval_loss'

In [ ]:
#save trained model. it only saves the adapter part
model_id = "distilbert-base-uncased-lora"
lora_model.save_pretrained(model_id)

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [ ]:
#inference with peft

#loading saved model
from peft import AutoPeftModelForCausalLM
lora_model = AutoPeftModelForCausalLM.from_pretrained("distilbert-base-uncased-lora")

from peft import PeftConfig

config = PeftConfig.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True
)

inference_model = PeftModel.from_pretrained(model, model_id)